In [ ]:
!pip install lbl2vec
!pip install gensim
!pip install sklearn
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 10.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyli

In [ ]:
from lbl2vec import Lbl2Vec
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import strip_tags
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive

/content/drive/My Drive


In [ ]:
ag_train_uk = pd.read_csv('train_uk.csv',sep=',',header=None, names=['title','text','tags','target'])
ag_test_uk = pd.read_csv('test_uk.csv',sep=',',header=None, names=['title','text','tags','target'])
labels_uk = pd.read_csv('labels_uk.csv',sep=';')

# split keywords by separator and save them as array
labels_uk['keywords'] = labels_uk['keywords'].apply(lambda x: x.split(' '))

# convert description keywords to lowercase
labels_uk['keywords'] = labels_uk['keywords'].apply(lambda description_keywords: [keyword.lower() for keyword in description_keywords])

# get number of keywords for each class
labels_uk['number_of_keywords'] = labels_uk['keywords'].apply(lambda row: len(row))

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
labels_uk

,class_index,class_name,keywords,number_of_keywords
0,1,політика,"[вибори, держава, президент, поліція, політика, безпека, зброя, війна, конфлікт, росія, китай, міністр, порошенко, зеленський, сша, росії, президента, ради, рада, нато, обсє, криму, донбас, донбасі, партії, виборів, україни, зброї, сирії]",29
1,2,спорт,"[футбол, спорт, ліга, чемпіонат, квитки, баскетбол, гра, ігри, тренування, збірна, команда, кубок, перемога, поразка, турнір, гол, тренер, матч, матчі, матчу, сезон, чемпіон, чемпіони, чемпіонату, чемпіонів, ставку, ставка, клуб, гравець, динамо, шахтар, ліги, команди, збірної, манчестер]",35
2,3,бізнес,"[компанія, нафта, ринок, покупець, бізнес, продукт, ціна, акція, акції, криптовалюта, ціни, прибуток, інвестор, млрд, ринку, зростання, курс, банку, ціни, газ, газу, гривні, долар, долара]",24
3,4,технології,"[лабораторія, компютер, технологія, google, microsoft, apple, samsung, інтернет, забезпечення, програма, космос, розробка, розробник, дослідження, windows, користувач, браузер, вчені, вчений, nasa, iphone, spacex, смартфон, android, xiaomi]",25
4,5,новини,"[місто, смерть, помер, новини, новина, люди, кіно, коронавірус, пандемія, народження, фільм, музика, пісня, євробачення, співак, область, випадків, водій, лікарні, чоловік, жінка, синоптик, дснс]",23


In [ ]:
import textwrap

labels = labels_uk.copy()

def split_keywords(keywords, width=50):
    text = ', '.join(keywords)
    return textwrap.fill(text, width=width, break_long_words=False)

labels['keywords'] = labels['keywords'].apply(lambda x: split_keywords(x, width=50))

from IPython.display import HTML

labels.rename(columns={'keywords': 'ключові слова', 'class_name': 'мітка', 'number_of_keywords': 'кількість ключових слів'}, inplace=True)

html = labels.to_html(escape=False).replace('\\n', '<br>')

HTML(html)

,class_index,мітка,ключові слова,кількість ключових слів
0,1,політика,"вибори, держава, президент, поліція, політика,безпека, зброя, війна, конфлікт, росія, китай,міністр, порошенко, зеленський, сша, росії,президента, ради, рада, нато, обсє, криму, донбас,донбасі, партії, виборів, україни, зброї, сирії",29
1,2,спорт,"футбол, спорт, ліга, чемпіонат, квитки, баскетбол,гра, ігри, тренування, збірна, команда, кубок,перемога, поразка, турнір, гол, тренер, матч,матчі, матчу, сезон, чемпіон, чемпіони,чемпіонату, чемпіонів, ставку, ставка, клуб,гравець, динамо, шахтар, ліги, команди, збірної,манчестер",35
2,3,бізнес,"компанія, нафта, ринок, покупець, бізнес, продукт,ціна, акція, акції, криптовалюта, ціни, прибуток,інвестор, млрд, ринку, зростання, курс, банку,ціни, газ, газу, гривні, долар, долара",24
3,4,технології,"лабораторія, компютер, технологія, google,microsoft, apple, samsung, інтернет, забезпечення,програма, космос, розробка, розробник,дослідження, windows, користувач, браузер, вчені,вчений, nasa, iphone, spacex, смартфон, android,xiaomi",25
4,5,новини,"місто, смерть, помер, новини, новина, люди, кіно,коронавірус, пандемія, народження, фільм, музика,пісня, євробачення, співак, область, випадків,водій, лікарні, чоловік, жінка, синоптик, дснс",23


In [ ]:
# add data set type column
ag_train_uk['data_set_type'] = 'train'
ag_test_uk['data_set_type'] = 'test'

# concat train and test data
ag_full_corpus_uk = pd.concat([ag_train_uk,ag_test_uk]).reset_index(drop=True)
# ag_full_corpus_uk = ag_full_corpus_uk.sample(frac=0.2, random_state=42)

In [ ]:
ag_full_corpus_uk.head()

,title,text,tags,target,data_set_type
117641,Міг закінчити кар'єру. Український коментатор розповів про травму Корнієнка,"Український коментатор Кирило Круторогов поділився думкою про травму 21-річного захисника «Шахтаря» Віктора Корнієнка, яку той отримав у матчі 9-го туру УПЛ проти київського «Динамо» (3:0). «Жахлива новина. Після такого фолу Віктор міг завершити кар'єру. Що для молодого хлопця, який чудово проявляє себе, важко ще й психологічно. Здоров'я та сил, Вікторе Корнієнко», — написав Круторогов. Раніше повідомлялося, що через травму Віктор вибув мінімум до кінця року.",Футбол|Україна. Прем'єр Ліга|Кирило Круторогов|Віктор Корнієнко|Шахтар Донецьк|травма|Динамо Київ|Динамо - Шахтар,спорт,train
139413,Потрібна субмарина. Астрономам вдалося дізнатися глибину моря на супутнику Сатурна,"Дані, зібрані зондом NASA Кассіні, дозволили вченим оцінити глибину Кракенового моря — найбільшого метанового моря на супутнику Сатурна Титані.Це море довжиною приблизно 1000 км більше всіх п’яти Великих озер Північної Америки разом узятих і порівняно з Каспійським морем. Кракенове море містить близько 80% рідин на поверхні Титана, пояснили астрономи. Титан — єдиний супутник у Сонячній системі, у якого є атмосфера. Це свого роду ковдра, яке покриває супутник. Вона приховує складну гідравлічну систему на поверхні, але замість рідкої води річки, озера і моря на Титані складаються з маслянистого чорного метану. Тепер вчені на чолі з Валеріо Поджіалі, науковим співробітником Корнельського центру астрофізики і планетології проаналізували всі дані місії Кассіні і вважають, що найглибше море Титана приблизно 0,3 км в найнижчій точці. Дослідники розробили акуратний метод визначення глибини морів Титана, який включає в себе вимір різниці між часом, який потрібен радару для відображення від поверхні моря і від морського дна. Цей метод допомагає оцінити глибину моря, але дослідники повинні зробити певні припущення про щільність рідин на Титані і про те, як швидко радіохвилі проходять через них. Використовуючи цю техніку, команда виміряла глибину Мурени, північного гирла Кракенового моря, яка, як вони виявили, становить 85 метрів. Швидкість поглинання радіолокаційних хвиль передбачає, що рідина в цій частині моря складається з 70% метану, 16% азоту і 14% етану. Сканування основної частини Кракенового моря було менш переконливим. Як пишуть автори в дослідженні, зонд NASA не виявлено «ніяких доказів відображення сигналів від морського дна, що свідчить про те, що рідина надто глибока, щоб радіохвилі Кассіні могли проникнути». Проте, якщо рідина в цій частині моря схожа за складом на рідину, виявлену в Мурені, то вона повинна бути глибше 100 метрів і, можливо, досягати 300 метрів. Поджіалі сподівається, що одного разу на Титан може бути відправлений ​​підводний човен-робот, щоб досліджувати море або будь-яку іншу водойму. Фактично, він вважає нове дослідження кроком в цьому напрямку.«Завдяки нашим вимірам вчені тепер можуть визначати щільність рідини з більш високою точністю і, отже, краще калібрувати гідролокатор на борту майбутньої роботизованого підводного човна», — пояснив Поджіалі. Раніше НВ писав, що астрономи виявили в атмосфері супутника Сатурна Титана ціклопропеніліден (C3H2) — надзвичайно рідкісну молекулу на основі вуглецю. Ця молекула настільки активна (вступає в хімічні реакції), що може існувати на Землі тільки в лабораторних умовах, підкреслили дослідники. Фактично, це настільки рідкісне поєднання, що його ніколи раніше не виявляли в атмосфері Землі, в Сонячній системі або де-небудь ще. Єдине інше місце, де ціклопропеніліден може залишатися стабільним, — це холодна порожнеча міжзоряного простору. Але ціклопропеніліден може стати будівельним блоком для більш складних органічних молекул, які одного дня можуть привести до життя.",Космос|Сатурн|Титан,технології,test
5170,У Харкові 84-річному державному зраднику дали 12 років тюрми,"Про це повідомляє прес-служба прокуратури Харківської області. Прокуратура довела, що чоловік збирав відомості, я

In [ ]:
# doc: document text string
# returns tokenized document
# strip_tags removes meta tags from the text
# simple preprocess converts a document into a list of lowercase tokens, ignoring tokens that are too short or too long
# simple preprocess also removes numerical values as well as punktuation characters
def tokenize(doc):
    return simple_preprocess(strip_tags(doc.replace("'", "")), min_len=2, max_len=15)

# tokenize and tag documents combined title + text for Lbl2Vec training
ag_full_corpus_uk['tagged_docs'] = ag_full_corpus_uk.apply(lambda row: TaggedDocument(tokenize(row['title'] + '. ' + row['text']), [str(row.name)]), axis=1)

In [ ]:
def count_words(tagged_doc):
  return len(tagged_doc.words)

result = ag_full_corpus_uk[ag_full_corpus_uk['tagged_docs'].apply(lambda x: count_words(x)) > 50]

In [ ]:
ag_full_corpus_uk = result

In [ ]:
# add doc_key column
ag_full_corpus_uk['doc_key'] = ag_full_corpus_uk.index.astype(str)

# add class_name column
ag_full_corpus_uk = ag_full_corpus_uk.merge(labels_uk, left_on='target', right_on='class_name', how='left').drop(['keywords', 'tags'], axis=1)

ag_full_corpus_uk.head()

,title,text,target,data_set_type,tagged_docs,doc_key,class_index,class_name,number_of_keywords
0,Міг закінчити кар'єру. Український коментатор розповів про травму Корнієнка,"Український коментатор Кирило Круторогов поділився думкою про травму 21-річного захисника «Шахтаря» Віктора Корнієнка, яку той отримав у матчі 9-го туру УПЛ проти київського «Динамо» (3:0). «Жахлива новина. Після такого фолу Віктор міг завершити кар'єру. Що для молодого хлопця, який чудово проявляє себе, важко ще й психологічно. Здоров'я та сил, Вікторе Корнієнко», — написав Круторогов. Раніше повідомлялося, що через травму Віктор вибув мінімум до кінця року.",спорт,train,"([міг, закінчити, карєру, український, коментатор, розповів, про, травму, корнієнка, український, коментатор, кирило, круторогов, поділився, думкою, про, травму, річного, захисника, шахтаря, віктора, корнієнка, яку, той, отримав, матчі, го, туру, упл, проти, київського, динамо, жахлива, новина, після, такого, фолу, віктор, міг, завершити, карєру, що, для, молодого, хлопця, який, чудово, проявляє, себе, важко, ще, психологічно, здоровя, та, сил, вікторе, корнієнко, написав, круторогов, раніше, повідомлялося, що, через, травму, віктор, вибув, мінімум, до, кінця, року], [117641])",117641,2,спорт,35
1,Потрібна субмарина. Астрономам вдалося дізнатися глибину моря на супутнику Сатурна,"Дані, зібрані зондом NASA Кассіні, дозволили вченим оцінити глибину Кракенового моря — найбільшого метанового моря на супутнику Сатурна Титані.Це море довжиною приблизно 1000 км більше всіх п’яти Великих озер Північної Америки разом узятих і порівняно з Каспійським морем. Кракенове море містить близько 80% рідин на поверхні Титана, пояснили астрономи. Титан — єдиний супутник у Сонячній системі, у якого є атмосфера. Це свого роду ковдра, яке покриває супутник. Вона приховує складну гідравлічну систему на поверхні, але замість рідкої води річки, озера і моря на Титані складаються з маслянистого чорного метану. Тепер вчені на чолі з Валеріо Поджіалі, науковим співробітником Корнельського центру астрофізики і планетології проаналізували всі дані місії Кассіні і вважають, що найглибше море Титана приблизно 0,3 км в найнижчій точці. Дослідники розробили акуратний метод визначення глибини морів Титана, який включає в себе вимір різниці між часом, який потрібен радару для відображення від поверхні моря і від морського дна. Цей метод допомагає оцінити глибину моря, але дослідники повинні зробити певні припущення про щільність рідин на Титані і про те, як швидко радіохвилі проходять через них. Використовуючи цю техніку, команда виміряла глибину Мурени, північного гирла Кракенового моря, яка, як вони виявили, становить 85 метрів. Швидкість поглинання радіолокаційних хвиль передбачає, що рідина в цій частині моря складається з 70% метану, 16% азоту і 14% етану. Сканування основної частини Кракенового моря було менш переконливим. Як пишуть автори в дослідженні, зонд NASA не виявлено «ніяких доказів відображення сигналів від морського дна, що свідчить про те, що рідина надто глибока, щоб радіохвилі Кассіні могли проникнути». Проте, якщо рідина в цій частині моря схожа за складом на рідину, виявлену в Мурені, то вона повинна бути глибше 100 метрів і, можливо, досягати 300 метрів. Поджіалі сподівається, що одного разу на Титан може бути відправлений ​​підводний човен-робот, щоб досліджувати море або будь-яку іншу водойму. Фактично, він вважає нове дослідження кроком в цьому напрямку.«Завдяки нашим вимірам вчені тепер можуть визначати щільність рідини з більш високою точністю і, отже, краще калібрувати гідролокатор на борту майбутньої роботизованого підводного човна», — пояснив Поджіалі. Раніше НВ писав, що астрономи виявили в атмосфері супутника Сатурна Титана ціклопропеніліден (C3H2) — надзвичайно рідкісну молекулу на основі вуглецю. Ця молекула настільки активна (вступає в хімічні реакції), що може існувати на Землі тільки в лабораторних умовах, підкреслили дослідники. Фактично, це настільки рідкісне поєднання, що його ніколи

In [ ]:
# init model with parameters
lbl2vec_model_uk = Lbl2Vec(keywords_list=list(labels_uk['keywords']), tagged_documents=ag_full_corpus_uk['tagged_docs'][ag_full_corpus_uk['data_set_type']=='train'], label_names=list(labels_uk['class_name']), similarity_threshold=0.30, min_num_docs=100, epochs=5)

In [ ]:
# train model
lbl2vec_model_uk.fit()

2024-06-03 01:18:40,801 - Lbl2Vec - INFO - Train document and word embeddings
INFO:Lbl2Vec:Train document and word embeddings
2024-06-03 01:25:50,927 - Lbl2Vec - INFO - Train label embeddings
INFO:Lbl2Vec:Train label embeddings
2024-06-03 01:25:51,063 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: баскетбол чемпіони
2024-06-03 01:25:51,195 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: браузер
2024-06-03 01:25:51,267 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: музика пісня євробачення


In [ ]:
# predict similarity scores
model_docs_lbl_similarities_uk = lbl2vec_model_uk.predict_model_docs()

2024-06-03 01:27:57,361 - Lbl2Vec - INFO - Get document embeddings from model
INFO:Lbl2Vec:Get document embeddings from model
2024-06-03 01:27:57,430 - Lbl2Vec - INFO - Calculate document<->label similarities
INFO:Lbl2Vec:Calculate document<->label similarities


In [ ]:
model_docs = model_docs_lbl_similarities_uk.copy()

model_docs.rename(columns={'most_similar_label': 'найбільш схожа мітка', 'highest_similarity_score': 'найбільша схожість', 'doc_key': 'номер документу'}, inplace=True)

model_docs.head()

,номер документу,найбільш схожа мітка,найбільша схожість,політика,спорт,бізнес,технології,новини
0,117641,спорт,0.717982,0.320992,0.717982,0.173709,0.176611,0.418978
1,5170,політика,0.538677,0.538677,0.100636,0.399076,0.268880,0.447723
2,113461,політика,0.600106,0.600106,0.203562,0.326363,0.318601,0.492845
3,16869,спорт,0.720440,0.331702,0.720440,0.264004,0.275616,0.392387
4,48454,технології,0.568946,0.181775,0.212646,0.296457,0.568946,0.269346


In [ ]:
# merge DataFrames to compare the predicted and true topic labels
evaluation_train = model_docs_lbl_similarities_uk.merge(ag_full_corpus_uk[ag_full_corpus_uk['data_set_type']=='train'], left_on='doc_key', right_on='doc_key')

In [ ]:
y_true_train = evaluation_train['class_name']
y_pred_train = evaluation_train['most_similar_label']
print('Accuracy:',accuracy_score(y_true_train, y_pred_train))

Accuracy: 0.7707931668385419


In [ ]:
mistakes_train = evaluation_train[evaluation_train['class_name'] != evaluation_train['most_similar_label']]
mistakes_train[:10]

,doc_key,most_similar_label,highest_similarity_score,політика,спорт,бізнес,технології,новини,title,text,target,data_set_type,tagged_docs,class_index,class_name,number_of_keywords
7,61371,бізнес,0.372271,0.227183,0.130069,0.372271,0.181276,0.221924,"Після аварійних зупинок ТЕС електроенергія в Україні подорожчала на 18%, - ""Наші гроші""","Про це пишуть ""Наші гроші"". Так, згідно з даними ""Оператора ринку"", за вісім днів лютого ціна електроенергії на ""ринку на добу наперед"" підстрибнула на 17,6% у порівнянні з середньою ціною за січень і склала, 1719,7 грн за МВт-год. ""Таке зростання ціни відбулося на фоні дефіцитів вугілля на складах ТЕС та аварійних зупинок енергоблоків ТЕС"", - пише видання. Зазначається, що минулого тижня Національна комісія, що здійснює державне регулювання у сфері енергетики та комунальних послуг (НКРЕКП) заявила, що проводить перевірки теплових електростанцій на предмет забезпечення паливом. Крім того, за даними регулятора, щоденне зменшення запасів вугілля загрожує стабільній роботі енергоблоків ТЕС, що може спровокувати виникнення загальносистемної аварії в об'єднаній енергосистемі України та введення в дію графіків аварійного припинення електропостачання споживачам. У виданні також нагадали, що минулого тижня відбулась серія аварійних зупинок енергоблоків ТЕС. Зокрема, 2 лютого відбулась аварійна зупинка блоків на Запорізькій ТЕС та Курахівській ТЕС (обидві належать ДТЕК), після чого була отримана аварійна допомога з Білорусі. Ввечері 3 лютого внаслідок аварії була повністю зупинена Запорізька ТЕС (ДТЕК), а через два дні там же аварійно був зупинений блок №1 Запорізької ТЕС. 6 лютого на електростанціях ДТЕК було виведено у аварійний ремонт ще два енергоблоки: на Ладижинській та Добротвірській ТЕС. За інформацією видання, всього станом на 7 лютого у аварійному ремонті перебувало дев'ять енергоблоків: сім енергоблоків ТЕС компанії ДТЕК та по одному у Центренерго та Донбасенерго. Ще шість енергоблоків не працювали через відсутність палива: два блоки ДТЕК. три блоки Центренерго та один блок Київської ТЕЦ-5. Дефіцити електроенергії покривалися імпортом із Білорусі та Росії. Зазначається, що близько 70% потужності теплової генерації контролює компанія ДТЕК Ріната Ахметова.",новини,train,"([після, аварійних, зупинок, тес, електроенергія, україні, подорожчала, на, наші, гроші, про, це, пишуть, наші, гроші, так, згідно, даними, оператора, ринку, за, вісім, днів, лютого, ціна, електроенергії, на, ринку, на, добу, наперед, підстрибнула, на, порівнянні, середньою, ціною, за, січень, склала, грн, за, мвт, год, таке, зростання, ціни, відбулося, на, фоні, дефіцитів, вугілля, на, складах, тес, та, аварійних, зупинок, енергоблоків, тес, пише, видання, зазначається, що, минулого, тижня, національна, комісія, що, здійснює, державне, регулювання, сфері, енергетики, та, комунальних, послуг, нкрекп, заявила, що, проводить, перевірки, теплових, електростанцій, на, предмет, забезпечення, паливом, крім, того, за, даними, регулятора, щоденне, зменшення, запасів, вугілля, загрожує, стабільній, роботі, енергоблоків, ...], [61371])",5,новини,15
17,67233,новини,0.516780,0.453076,0.313433,0.197945,0.219053,0.516780,Макаревич відповів на цькування за проукраїнську позицію новою піснею,"В лірико-іронічній формі Макаревич співає про цькування, яку пережив, і про реакцію на неї деякої частини суспільства: ""Ну що ти розспівався, любий? Ну хто ж тебе просив?"" Тиск на Макаревича почався після того, як 12 серпня він виступив перед українськими біженцями в Святогорську. Після цього концерту депутат Держдуми Євген Федоров запропонував позбавити Макаревича всіх державних нагород. Лідер ""Машини часу"" за свою позицію зіткнувся з відміною концертів в російських містах.",політика,train,"([макаревич, відповів, на, цькування, за, проукраїнську, позицію, новою, піснею, лірико, іронічній, формі, макаревич, співає, про, цькування, яку, пережив, про, реакцію, на, неї, деякої, частини, суспільства, ну, що, ти, розспівався, любий, ну, хто, тебе

In [ ]:
new_docs_lbl_similarities_uk = lbl2vec_model_uk.predict_new_docs(tagged_docs=ag_full_corpus_uk['tagged_docs'][ag_full_corpus_uk['data_set_type']=='test'])

2024-06-03 01:29:39,281 - Lbl2Vec - INFO - Calculate document embeddings
INFO:Lbl2Vec:Calculate document embeddings
2024-06-03 01:29:52,768 - Lbl2Vec - INFO - Calculate document<->label similarities
INFO:Lbl2Vec:Calculate document<->label similarities


In [ ]:
evaluation_test = new_docs_lbl_similarities_uk.merge(ag_full_corpus_uk[ag_full_corpus_uk['data_set_type']=='test'], left_on='doc_key', right_on='doc_key')

y_true_test = evaluation_test['class_name'][1:]
y_pred_test = evaluation_test['most_similar_label'][1:]
print('Accuracy:',accuracy_score(y_true_test, y_pred_test))

Accuracy: 0.7787079162875341


In [ ]:
mistakes_test = evaluation_test[evaluation_test['class_name'] != evaluation_test['most_similar_label']]
mistakes_test[:10]

,doc_key,most_similar_label,highest_similarity_score,політика,спорт,бізнес,технології,новини,title,text,target,data_set_type,tagged_docs,class_index,class_name,number_of_keywords
1,148462,новини,0.510417,0.213070,0.411605,0.130471,0.221204,0.510417,Російська гімнастка: «Пишуть про суїцид?Я на кухні порізалася! Ножі гострі»,"Зіркова 21-річна російська гімнастка Александра Солдатова, багаторазова чемпіонка світу і Європи, потрапила до лікарні в Москві. Раніше повідомлялося, що причиною могла стати спроба суїциду. Нібито спортсменка порізала собі вени, але потім злякалася і звернулася за допомогою до медиків. Сама гімнастика після деякого мовчання пояснила ситуацію так: «Я вчора їхала на тренування, вранці готувала сніданок, порізалася. Таке буває, у всіх буває. Ножі гострі, я гімнастка, на кухні стою не так давно. Їду на тренування і відчуваю – рани краще заклеїти і обробити професійно, щоб не займатися самолікуванням. Заїхала в лікарню, попередила тренерів, що я трохи спізнююся, і на цьому вся історія закінчилася. А тут пишуть такий кошмар: вени, порізалась, який жах. Насправді, у мене просто немає слів. Зі мною все прекрасно, жива-здорова, буду на Гран-прі, буду виступати в гала-концерті. Все супер, зараз поїду на тренування». Посмотреть эту публикацию в Instagram Без комментариев! И НИКОГДА НЕ ВЕРЬТЕ ПРЕССЕ!!! Публикация от AV (@love_shumik) 6 Фев 2020 в 4:41 PST",спорт,test,"([російська, гімнастка, пишуть, про, суїцид, на, кухні, порізалася, ножі, гострі, зіркова, річна, російська, гімнастка, александра, солдатова, багаторазова, чемпіонка, світу, європи, потрапила, до, лікарні, москві, раніше, повідомлялося, що, причиною, могла, стати, спроба, суїциду, нібито, спортсменка, порізала, собі, вени, але, потім, злякалася, звернулася, за, допомогою, до, медиків, сама, гімнастика, після, деякого, мовчання, пояснила, ситуацію, так, вчора, їхала, на, тренування, вранці, готувала, сніданок, порізалася, таке, буває, всіх, буває, ножі, гострі, гімнастка, на, кухні, стою, не, так, давно, їду, на, тренування, відчуваю, рани, краще, заклеїти, обробити, професійно, щоб, не, займатися, самолікуванням, заїхала, лікарню, попередила, тренерів, що, трохи, спізнююся, на, цьому, вся, історія, закінчилася, тут, ...], [148462])",2,спорт,23
2,141644,новини,0.435085,0.404043,0.226538,0.222946,0.201887,0.435085,Польські кіномитці масово підтримали Сенцова,"Про це повідомляє Еспресо.TV з посиланням на ""Радіо Свобода"". Понад 700 присутніх у залі кіномитців висловилися на підтримку українського кінорежисера Олега Сенцова, ув’язненого в Росії. На сцені «Театру Польського», де вручали нагороди, висвітлили кінокадри з російського суду над Сенцовим. Голова Польської кіноакадемії Даріуш Яблонський та директор Європейської кіноакадемії Маріон Дерінґ закликали владу Росії звільнити ув’язненого українського режисера. Яблонський звернувся зі сцени безпосередньо до Путіна зі словами: ""Пане Путін, звільніть Сенцова! Ми не залишимо вас у спокої, кіномитці будуть нагадувати вам про Олега доти, доки він буде перебувати у в’язниці"". Усі учасники урочистих заходів піднесли вгору картки з написом ""Свободу Олегу Сенцову!"". Podnieśliśmy wszyscy razem kartki w geście buntu wobec uwięzienia Oleg Sentsov. Uwolnić Olega Senstova!#PNF #Orły2016 #UwolnićOlegaSentsova Опубліковано Polskie Nagrody Filmowe ORŁY 7 березень 2016 р. Нагадаємо, акція на підтримку Олега Сенцова під час вручення «Орлів» відбулася вже вдруге. Із закликом звільнити ув’язненого українського режисера виступили відомі польські кіномитці, зокрема Анджей Вайда, Агнешка Голланд, Кшиштоф Зануссі, Даніель Ольбрихський та інші.",політика,test,"([польські, кіномитці, масово, підтримали, сенцова, про, це, повідомляє, еспресо, tv, посиланням, на, радіо, свобода, понад, присутніх, залі, кіномитців, висловилися, на, підтримку, українського, кінорежисера, олега, сенцова, ув, язненого, росії, на, сцені, театру, польського, де, вручали, нагороди, висвітлили, кінокадри, російського, суду, над, сенцовим, голова, польськ

In [ ]:
ag_full_corpus_uk['full_doc'] = ag_full_corpus_uk['title'] + '. ' + ag_full_corpus_uk['text']
ag_full_corpus_uk[:10]

,title,text,target,data_set_type,tagged_docs,doc_key,class_index,class_name,number_of_keywords,full_doc
0,Міг закінчити кар'єру. Український коментатор розповів про травму Корнієнка,"Український коментатор Кирило Круторогов поділився думкою про травму 21-річного захисника «Шахтаря» Віктора Корнієнка, яку той отримав у матчі 9-го туру УПЛ проти київського «Динамо» (3:0). «Жахлива новина. Після такого фолу Віктор міг завершити кар'єру. Що для молодого хлопця, який чудово проявляє себе, важко ще й психологічно. Здоров'я та сил, Вікторе Корнієнко», — написав Круторогов. Раніше повідомлялося, що через травму Віктор вибув мінімум до кінця року.",спорт,train,"([міг, закінчити, карєру, український, коментатор, розповів, про, травму, корнієнка, український, коментатор, кирило, круторогов, поділився, думкою, про, травму, річного, захисника, шахтаря, віктора, корнієнка, яку, той, отримав, матчі, го, туру, упл, проти, київського, динамо, жахлива, новина, після, такого, фолу, віктор, міг, завершити, карєру, що, для, молодого, хлопця, який, чудово, проявляє, себе, важко, ще, психологічно, здоровя, та, сил, вікторе, корнієнко, написав, круторогов, раніше, повідомлялося, що, через, травму, віктор, вибув, мінімум, до, кінця, року], [117641])",117641,2,спорт,23,"Міг закінчити кар'єру. Український коментатор розповів про травму Корнієнка. Український коментатор Кирило Круторогов поділився думкою про травму 21-річного захисника «Шахтаря» Віктора Корнієнка, яку той отримав у матчі 9-го туру УПЛ проти київського «Динамо» (3:0). «Жахлива новина. Після такого фолу Віктор міг завершити кар'єру. Що для молодого хлопця, який чудово проявляє себе, важко ще й психологічно. Здоров'я та сил, Вікторе Корнієнко», — написав Круторогов. Раніше повідомлялося, що через травму Віктор вибув мінімум до кінця року."
1,Потрібна субмарина. Астрономам вдалося дізнатися глибину моря на супутнику Сатурна,"Дані, зібрані зондом NASA Кассіні, дозволили вченим оцінити глибину Кракенового моря — найбільшого метанового моря на супутнику Сатурна Титані.Це море довжиною приблизно 1000 км більше всіх п’яти Великих озер Північної Америки разом узятих і порівняно з Каспійським морем. Кракенове море містить близько 80% рідин на поверхні Титана, пояснили астрономи. Титан — єдиний супутник у Сонячній системі, у якого є атмосфера. Це свого роду ковдра, яке покриває супутник. Вона приховує складну гідравлічну систему на поверхні, але замість рідкої води річки, озера і моря на Титані складаються з маслянистого чорного метану. Тепер вчені на чолі з Валеріо Поджіалі, науковим співробітником Корнельського центру астрофізики і планетології проаналізували всі дані місії Кассіні і вважають, що найглибше море Титана приблизно 0,3 км в найнижчій точці. Дослідники розробили акуратний метод визначення глибини морів Титана, який включає в себе вимір різниці між часом, який потрібен радару для відображення від поверхні моря і від морського дна. Цей метод допомагає оцінити глибину моря, але дослідники повинні зробити певні припущення про щільність рідин на Титані і про те, як швидко радіохвилі проходять через них. Використовуючи цю техніку, команда виміряла глибину Мурени, північного гирла Кракенового моря, яка, як вони виявили, становить 85 метрів. Швидкість поглинання радіолокаційних хвиль передбачає, що рідина в цій частині моря складається з 70% метану, 16% азоту і 14% етану. Сканування основної частини Кракенового моря було менш переконливим. Як пишуть автори в дослідженні, зонд NASA не виявлено «ніяких доказів відображення сигналів від морського дна, що свідчить про те, що рідина надто глибока, щоб радіохвилі Кассіні могли проникнути». Проте, якщо рідина в цій частині моря схожа за складом на рідину, виявлену в Мурені, то вона повинна бути глибше 100 метрів і, можливо, досягати 300 метрів. Поджіалі сподівається, що одного разу на Титан може бути відправлений ​​підводний човен-робот, щоб досліджувати море або будь-яку іншу водойму. Фактично, він вважає нове дослідження кроком в цьому напрямку.«За

In [ ]:
lbl_dict = {'політика':1, 'спорт':2, 'бізнес':3, 'технології':4, 'новини':5}
full_evaluation_df = pd.concat([evaluation_train, evaluation_test], ignore_index=True)
full_evaluation_df['label_index'] = full_evaluation_df['most_similar_label'].apply(lambda row: lbl_dict[row])

In [ ]:
full_evaluation_df

,doc_key,most_similar_label,highest_similarity_score,політика,спорт,бізнес,технології,новини,title,text,target,data_set_type,tagged_docs,class_index,class_name,number_of_keywords,label_index
0,117641,спорт,0.644971,0.269221,0.644971,0.142862,0.145387,0.443285,Міг закінчити кар'єру. Український коментатор розповів про травму Корнієнка,"Український коментатор Кирило Круторогов поділився думкою про травму 21-річного захисника «Шахтаря» Віктора Корнієнка, яку той отримав у матчі 9-го туру УПЛ проти київського «Динамо» (3:0). «Жахлива новина. Після такого фолу Віктор міг завершити кар'єру. Що для молодого хлопця, який чудово проявляє себе, важко ще й психологічно. Здоров'я та сил, Вікторе Корнієнко», — написав Круторогов. Раніше повідомлялося, що через травму Віктор вибув мінімум до кінця року.",спорт,train,"([міг, закінчити, карєру, український, коментатор, розповів, про, травму, корнієнка, український, коментатор, кирило, круторогов, поділився, думкою, про, травму, річного, захисника, шахтаря, віктора, корнієнка, яку, той, отримав, матчі, го, туру, упл, проти, київського, динамо, жахлива, новина, після, такого, фолу, віктор, міг, завершити, карєру, що, для, молодого, хлопця, який, чудово, проявляє, себе, важко, ще, психологічно, здоровя, та, сил, вікторе, корнієнко, написав, круторогов, раніше, повідомлялося, що, через, травму, віктор, вибув, мінімум, до, кінця, року], [117641])",2,спорт,23,2
1,5170,політика,0.343529,0.343529,0.120108,0.213756,0.163911,0.255744,У Харкові 84-річному державному зраднику дали 12 років тюрми,"Про це повідомляє прес-служба прокуратури Харківської області. Прокуратура довела, що чоловік збирав відомості, які належать науковим установам та становлять державну таємницю. У подальшому інформацію планувалося передати представникам Росії. Зловмисник у поле зору СБУ потрапив у серпні 2017 року. Він збирав відомості щодо державного оборонного замовлення, яке виконується одним із наукових центрів міста на підставі угод з Міністерством оборони України. Суд призначив чоловіку покарання у вигляді 12 років позбавлення волі. Наразі вирок суду ще не набув чинності, оскільки триває час на апеляційне оскарження. Зараз засуджений перебуває під вартою. Як повідомляє ""Українська правда"", засуджений - 84-річний харків'янин Мехті Логунов. Адвокат Логунова Максим Шаповалов розповів, що суд виніс вирок 30 липня. Тепер захист збирається оскаржувати це рішення. На подачу апеляції в нього є 30 днів. Повідомляється, що ще у 2017 році Логунов створив терористичну організацію, до якої залучив 18 харків'ян. Тоді його цікавили дані про радіоактивні відходи, де вони зберігаються. Загалом зрадник передавав до Москви інформація про військові об'єкти та кількість озброєння, особисті дані українських військових та патріотично налаштованих харків'ян. У січні 2018 року Логунова внесли до списку другого обміну в'язнів на Донбасі.",політика,train,"([харкові, річному, державному, зраднику, дали, років, тюрми, про, це, повідомляє, прес, служба, прокуратури, харківської, області, прокуратура, довела, що, чоловік, збирав, відомості, які, належать, науковим, установам, та, становлять, державну, таємницю, подальшому, інформацію, планувалося, передати, представникам, росії, зловмисник, поле, зору, сбу, потрапив, серпні, року, він, збирав, відомості, щодо, державного, оборонного, замовлення, яке, виконується, одним, із, наукових, центрів, міста, на, підставі, угод, міністерством, оборони, україни, суд, призначив, чоловіку, покарання, вигляді, років, позбавлення, волі, наразі, вирок, суду, ще, не, набув, чинності, оскільки, триває, час, на, апеляційне, оскарження, зараз, засуджений, перебуває, під, вартою, як, повідомляє, українська, правда, засуджений, річний, харківянин, мехті, логунов, адвокат, логунова, максим, ...], [5170])",1,політика,15,1
2,113461,політика,0.477450,0.477450,0.136872,0.271097,0.291671,0.399709,"У російському Слідкомі марять, що СБУ хоче арештувати засновника Google","Про це пише Еспресо.TV із посиланням на Twitter Маркіна. ""З

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
doc_topic_dists_vectorizer = CountVectorizer(analyzer='char')
doc_topic_dists = doc_topic_dists_vectorizer.fit_transform([str(x) for x in list(full_evaluation_df['label_index'])])

In [ ]:
import re
# concat title + descrpition to one text for each document
full_evaluation_df['full_text'] = full_evaluation_df.apply(lambda row: row['title'] + '. ' + row['text'], axis=1)
# Remove punctuation
full_evaluation_df['full_text_processed'] = full_evaluation_df['full_text'].map(lambda row: re.sub('[,\.!?\']', '', row))
# Convert the titles to lowercase
full_evaluation_df['full_text_processed'] = full_evaluation_df['full_text_processed'].map(lambda row: row.lower())
# split into word list
full_evaluation_df['full_text_processed'] = full_evaluation_df['full_text'].apply(lambda row: tokenize(row))
# remove stop words
full_evaluation_df['full_text_processed'] = full_evaluation_df['full_text_processed'].apply(lambda row: [word for word in row if len(word)>3])
# join word list to single string
full_evaluation_df['full_text_processed'] = full_evaluation_df['full_text_processed'].apply(lambda row: ' '.join(row))
# get doc lengths
full_evaluation_df['doc_length'] = full_evaluation_df['full_text_processed'].apply(lambda x: len(x.split(' ')))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:5: DeprecationWarning: invalid escape sequence '\.'
<>:5: DeprecationWarning: invalid escape sequence '\.'
<ipython-input-38-6cb7cb35a06b>:5: DeprecationWarning: invalid escape sequence '\.'
  full_evaluation_df['full_text_processed'] = full_evaluation_df['full_text'].map(lambda row: re.sub('[,\.!?\']', '', row))


In [ ]:
# group full articles (title + text) by classified label
docs_per_class = full_evaluation_df.groupby(['most_similar_label'], as_index=False).agg({'full_text_processed': ' '.join})
# create topic-word distribution with KxM shape, where K is number of topics and M is vocabulary size
topic_vectorizer = CountVectorizer()
topic_term_matrix = topic_vectorizer.fit_transform(list(docs_per_class['full_text_processed']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pyLDAvis
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { max-width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.output_area { max-width:100% !important; }</style>"))
display(HTML("<style>.input_area { max-width:100% !important; }</style>"))
pyLDAvis.enable_notebook()

# prepare pyLDAvis data
pyLDAvis_data = pyLDAvis.prepare(mds='pcoa', topic_term_dists=topic_term_matrix.toarray(), doc_topic_dists=doc_topic_dists.toarray(), doc_lengths=list(full_evaluation_df['doc_length']), vocab=topic_vectorizer.get_feature_names_out(), term_frequency=list(pd.DataFrame(topic_term_matrix.toarray()).sum()), n_jobs=1)

# parse topic names
parsedLDAtopics = pyLDAvis_data.topic_coordinates.reset_index().merge(docs_per_class.reset_index(), left_on='topic',right_on='index')[['topics','most_similar_label']]
parsedpyLDAvistopicsdict = dict(zip(list(parsedLDAtopics['topics']),list(parsedLDAtopics['most_similar_label'])))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [ ]:
# display pyLDAvis data
print('Parsed pyLDAvis topics to class names:',parsedpyLDAvistopicsdict)
pyLDAvis.display(pyLDAvis_data)

Parsed pyLDAvis topics to class names: {1: 'бізнес', 2: 'політика', 3: 'спорт', 4: 'новини', 5: 'технології'}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
